In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import os
import mlflow
import mlflow.pytorch
from torchvision import datasets , transforms
from torch.optim.lr_scheduler import StepLR

In [2]:
class Config:
    EPOCHS = 10
    BATCH_SIZE = 32
    LR = 0.01
    Device = "cpu"
    Gamma = 0.7
    seed =42
    Log_interval = 10
    Test_Batch_size = 1000
    Dry_run = True
    

In [3]:
config = Config()

In [18]:
torch.manual_seed(config.seed)

class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = nn.Dropout(0.25)
        self.dropout2 = nn.Dropout(0.5)
        self.fc1 = nn.Linear(9216, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
#         output = F.log_softmax(x, dim=1)
        return x

In [19]:
def train_(config, model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        pred = model.forward(data)
        loss = F.cross_entropy(pred, target)
        loss.backward()
        optimizer.step()
        if batch_idx % config.Log_interval == 0:
            print(f"Train Epoch: {epoch} [{batch_idx * len(data)}/{len(train_loader.dataset)} ({100.0 * batch_idx / len(train_loader):.0f}%)]\tLoss: {loss.item():.6f}")
            if config.Dry_run:
                break

In [20]:
def test(model, device, test_loader):
    pass

In [21]:
train_kwargs = {"batch_size": config.BATCH_SIZE}
test_kwargs = {"batch_size": config.Test_Batch_size}

In [22]:
transform = transforms.Compose(
    [transforms.ToTensor()]
)

In [11]:
train = datasets.MNIST("../data", train=True, download=True, transform=transform)
test = datasets.MNIST("../data", train=False, download=True, transform=transform)

train_loader = torch.utils.data.DataLoader(train, **train_kwargs)
test_loader = torch.utils.data.DataLoader(test, **test_kwargs)

100.0%


Extracting ../data\MNIST\raw\train-images-idx3-ubyte.gz to ../data\MNIST\raw



100.0%


Extracting ../data\MNIST\raw\train-labels-idx1-ubyte.gz to ../data\MNIST\raw



100.0%


Extracting ../data\MNIST\raw\t10k-images-idx3-ubyte.gz to ../data\MNIST\raw



100.0%

Extracting ../data\MNIST\raw\t10k-labels-idx1-ubyte.gz to ../data\MNIST\raw



In [23]:
train_kwargs

{'batch_size': 32}

In [25]:
model = ConvNet().to(config.Device)
scripted_model = torch.jit.script(model)
optimizer = torch.optim.Adam(model.parameters(), lr=config.LR)

scheduler = StepLR(optimizer, step_size=1, gamma=config.Gamma)

In [29]:
for i in train_loader:
    print(i[0],i[1])
    break

tensor([[[[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]]],


        [[[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]]],


        [[[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]]],


        ...,


        [[[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0.

In [26]:
for i in train_loader:
    print(i[0].shape,i[1].shape)
    break

torch.Size([32, 1, 28, 28]) torch.Size([32])


In [27]:
train_loader

In [32]:
# training loop

for epoch in range(1, config.EPOCHS + 1):
    train_(config, scripted_model, config.Device, train_loader, optimizer, epoch)
    scheduler.step()

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.309588
Train Epoch: 2 [0/60000 (0%)]	Loss: 6.650487
Train Epoch: 3 [0/60000 (0%)]	Loss: 2.544785
Train Epoch: 4 [0/60000 (0%)]	Loss: 2.300918
Train Epoch: 5 [0/60000 (0%)]	Loss: 2.219800
Train Epoch: 6 [0/60000 (0%)]	Loss: 2.227567
Train Epoch: 7 [0/60000 (0%)]	Loss: 2.229790
Train Epoch: 8 [0/60000 (0%)]	Loss: 2.210633
Train Epoch: 9 [0/60000 (0%)]	Loss: 2.212388
Train Epoch: 10 [0/60000 (0%)]	Loss: 2.199289


In [33]:
train_

<function __main__.train_(config, model, device, train_loader, optimizer, epoch)>

In [39]:
with mlflow.start_run() as run:
    mlflow.pytorch.log_model(model, "model")
    model_path = mlflow.get_artifact_uri("model")
    loaded_torch_model = mlflow.pytorch.load_model(model_path)
    model.eval()
    with torch.no_grad():
        test_datapoints, test_target = next(iter(test_loader))
        pred = model(test_datapoints[10].reshape((1,1,28,28)).to(config.Device))
#         print(test_datapoints)
        actual = test_target[10].item()
        predicted = torch.argmax(pred).item()
        print(f"actual: {actual}, predicted: {predicted}")

actual: 0, predicted: 2


In [36]:
print(test_datapoints[0].reshape((1,1,28,28)).shape)

torch.Size([1, 1, 28, 28])
